### Install the required libraries

In [ ]:
!pip install pandas fastparquet plotly

### Read Data

In [3]:
import pandas as pd

df = pd.read_csv("hackathon_data.csv", low_memory=False)

### Function to calculate customer churn and transaction frequency from a DataFrame based on transaction history and a specified inactivity threshold

In [19]:
import pandas as pd

def analyze_customer_churn(df, id_col='id', transaction_date_col='IslemTarih',
                           transaction_id_col='IslemID', membership_status_col='UyeDurum',
                           churn_days_threshold=(20, 90)):
    """
    Analyze customer churn from transaction data and categorize them into 'Retained', 'At Risk', or 'Churned'.
    Also includes the membership status of the first record for each customer.

    Parameters
    ----------
    df : pandas.DataFrame
        A DataFrame containing the transaction data.
    id_col : str, optional
        The name of the column that contains the customer ID. Default is 'id'.
    transaction_date_col : str, optional
        The name of the column that contains the transaction date. Default is 'IslemTarih'.
    transaction_id_col : str, optional
        The name of the column that contains the transaction ID. Default is 'IslemID'.
    membership_status_col : str, optional
        The name of the column that contains the membership status. Default is 'UyeDurumu'.
    churn_days_threshold : tuple, optional
        A tuple representing the lower and upper thresholds for days without transactions to 
        categorize customers as 'At Risk' or 'Churned'. Defaults to (20, 90), where:
        - 0-20 days since last transaction is 'Retained',
        - 21-89 days since last transaction is 'At Risk',
        - 90+ days since last transaction is 'Churned'.

    Returns
    -------
    pandas.DataFrame
        A DataFrame with churn analysis by customer, including churn status and initial membership status.

    Examples
    --------
    >>> df = pd.DataFrame({
    ...     'id': [1, 1, 2, 2, 3],
    ...     'IslemTarih': ['2021-01-01', '2021-01-02', '2021-01-02', '2021-02-01', '2021-03-01'],
    ...     'IslemID': [101, 102, 103, 104, 105],
    ...     'UyeDurumu': ['Silver', 'Silver', 'Gold', 'Gold', 'Platinum']
    ... })
    >>> result = analyze_customer_churn(df)
    >>> result
    """
    df[transaction_date_col] = pd.to_datetime(df[transaction_date_col])

    # Retrieve the first membership status for each customer
    membership_status = df.groupby(id_col)[membership_status_col].first().rename('MembershipStatus')

    transaction_dates = df.groupby(id_col)[transaction_date_col].agg(['min', 'max'])
    transaction_dates.columns = ['FirstTransactionDate', 'LastTransactionDate']

    latest_date = df[transaction_date_col].max()
    days_since_last_transaction = (latest_date - transaction_dates['LastTransactionDate']).dt.days

    # Determine churn status based on days since last transaction
    conditions = [
        days_since_last_transaction <= churn_days_threshold[0],
        days_since_last_transaction.between(churn_days_threshold[0] + 1, churn_days_threshold[1]),
        days_since_last_transaction >= churn_days_threshold[1] + 1
    ]
    choices = ['Retained', 'At Risk', 'Churned']
    churn_status = pd.cut(days_since_last_transaction, bins=[-1, churn_days_threshold[0], churn_days_threshold[1], float('inf')], labels=choices)

    churn_analysis = pd.concat([
        membership_status,
        transaction_dates,
        days_since_last_transaction.rename('DaysSinceLastTransaction'),
        churn_status.rename('ChurnStatus')
    ], axis=1).reset_index()

    return churn_analysis

### Show Result:

In [20]:
result = analyze_customer_churn(df)

result

,id,MembershipStatus,FirstTransactionDate,LastTransactionDate,DaysSinceLastTransaction,ChurnStatus
0,301002470,Aktif,2015-09-18 14:35:04,2023-10-26 10:47:00,0,Retained
1,301002583,Pasif,2014-09-25 10:08:52,2020-12-23 12:28:28,1037,Churned
2,301003354,Aktif,2022-11-21 18:05:00,2023-10-26 16:59:00,0,Retained
3,301006906,Pasif,2016-01-29 11:00:43,2022-09-12 18:30:00,408,Churned
4,301009366,Pasif,2019-06-27 17:31:14,2022-09-19 18:01:00,401,Churned
5,301009412,Pasif,2016-08-26 11:00:35,2023-10-03 17:56:28,22,At Risk
6,301009682,Aktif,2016-05-19 10:29:01,2023-10-26 16:39:00,0,Retained
7,301010389,Aktif,2016-06-15 15:12:25,2020-05-10 11:09:14,1264,Churned
8,301012873,Aktif,2018-12-07 12:52:00,2023-10-26 15:06:00,0,Retained
9,301014561,Aktif,2017-08-10 14:22:03,2023-10-26 16:55:00,0,Retained


--- 

## ⚠️ Customer Anomaly Notice ⚠️

### Summary of Anomalies

- 👤 **Customer ID:** `301026958`
- 📅 **Days Since Last Transaction:** `904`
- ✅ **Status According to Records:** *Active*

Despite no transactional activity for a considerable period, this customer's status is still marked as *active*, which is an unusual record that needs attention.

- 👤 **Customer ID:** `301010389`
- 📅 **Days Since Last Transaction:** `1264`
- ✅ **Status According to Records:** *Active*

For this customer, the gap since the last transaction is even more significant. Yet, similarly, the status is indicated as *active* (Aktif), suggesting a discrepancy that requires investigation.

---

In [8]:
result.to_csv("churn_analysis.csv", index=False)

In [21]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        39 non-null     int64         
 1   MembershipStatus          39 non-null     object        
 2   FirstTransactionDate      39 non-null     datetime64[ns]
 3   LastTransactionDate       39 non-null     datetime64[ns]
 4   DaysSinceLastTransaction  39 non-null     int64         
 5   ChurnStatus               39 non-null     category      
dtypes: category(1), datetime64[ns](2), int64(2), object(1)
memory usage: 1.8+ KB


---

# Column Descriptions

This document provides a brief description of each column in the dataset.

## Dataset Columns

- `id`:
  - **Description**: The unique identifier for the merchant (merchantID).

- `MembershipStatus`:
  - **Description**: The active status of the company within Ödeal's platform.

- `FirstTransactionDate`:
  - **Description**: The date and time of the first transaction made by the member merchant (customer).

- `LastTransactionDate`:
  - **Description**: The date and time of the last transaction made by the member merchant (customer).

- `DaysSinceLastTransaction`:
  - **Description**: The number of days that have passed since the member merchant's (customer's) last transaction.

- `ChurnStatus`:
  - **Description**: A column indicating the customer's discontinuation status of service or product. The values are determined based on the customer's last transaction date and are divided into three categories:
    - **"Retained"**: If 0-20 days have passed since the customer's last transaction, it signifies that the customer is still active and continuing to use the service.
    - **"At Risk"**: If 21-89 days have passed since the customer's last transaction, it indicates a potential risk of the customer discontinuing the service or product in the future.
    - **"Churned"**: If 90 or more days have passed since the customer's last transaction, it implies that the customer has discontinued the service or product and is no longer active, which is considered a customer loss.

---

In [22]:
result.ChurnStatus.value_counts() * 100 / len(result)

ChurnStatus
Retained    48.717949
Churned     41.025641
At Risk     10.256410
Name: count, dtype: float64

### Visualize the churn rate 

In [23]:
import pandas as pd
import plotly.express as px

def create_churn_rate_pie_chart(dataframe, churn_column='ChurnStatus', title='<b>Churn Rate</b>', show_fig=True):
    """
    Creates a pie chart visualizing churn rates using Plotly.

    Parameters
    ----------
    dataframe : DataFrame
        A pandas DataFrame containing the data.
    churn_column : str, optional
        The column name in dataframe that contains the churn statuses, by default 'Churn'.
    title : str, optional
        Title of the pie chart, by default '<b>Churn Rate</b>'.
    show_fig : bool, optional
        If True, the figure will be displayed in the browser, by default True.

    Returns
    -------
    figure : plotly.graph_objs._figure.Figure
        A plotly Figure object representing the pie chart.

    Examples
    --------
    >>> create_churn_rate_pie_chart(result_df, churn_column='Churn')
    This will create and display a pie chart with churn data taken from `result_df`.

    """
    dataframe['Churn_Label'] = dataframe[churn_column].replace({True: 'Churned', False: 'Retained'})
    
    churn_counts = dataframe.groupby('Churn_Label').size().reset_index(name='Count')
    
    churn_counts['Percentage'] = (churn_counts['Count'] / churn_counts['Count'].sum() * 100).apply(lambda x: f"{x:.2f}%")

    fig = px.pie(
        churn_counts,
        names='Churn_Label',
        values='Count',
        title=title,
        color_discrete_sequence=px.colors.sequential.Agsunset,
        hole=0.3
    )

    fig.update_layout(
        title_font_size=24,
        title_font_color="navy",
        title_font_family="Arial",
        title_x=0.5,
        legend_title_text='Status'
    )

    fig.update_traces(
        textinfo='percent+label',
        texttemplate='%{label}: %{customdata[0]}',
        hovertemplate='%{label}: %{percent:.2f}%',
        pull=[0.1 if label == 'Churned' else 0 for label in churn_counts['Churn_Label']],
        marker=dict(line=dict(color='#000000', width=2)),
        customdata=churn_counts[['Percentage']]
    )

    if show_fig:
        fig.show(renderer='browser')

    return fig

In [25]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning) 

create_churn_rate_pie_chart(result)